In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("lmsys-chatbot-arena/test.csv")

In [3]:
df

,id,prompt,response_a,response_b
0,136060,"[""I have three oranges today, I ate an orange ...","[""You have two oranges today.""]","[""You still have three oranges. Eating an oran..."
1,211333,"[""You are a mediator in a heated political deb...","[""Thank you for sharing the details of the sit...","[""Mr Reddy and Ms Blue both have valid points ..."
2,1233961,"[""How to initialize the classification head wh...","[""When you want to initialize the classificati...","[""To initialize the classification head when p..."


In [4]:
df = pd.read_csv("lmsys-chatbot-arena/train.csv")

In [5]:
df

,id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
0,30192,gpt-4-1106-preview,gpt-4-0613,"[""Is it morally right to try to have a certain...","[""The question of whether it is morally right ...","[""As an AI, I don't have personal beliefs or o...",1,0,0
1,53567,koala-13b,gpt-4-0613,"[""What is the difference between marriage lice...","[""A marriage license is a legal document that ...","[""A marriage license and a marriage certificat...",0,1,0
2,65089,gpt-3.5-turbo-0613,mistral-medium,"[""explain function calling. how would you call...","[""Function calling is the process of invoking ...","[""Function calling is the process of invoking ...",0,0,1
3,96401,llama-2-13b-chat,mistral-7b-instruct,"[""How can I create a test set for a very rare ...","[""Creating a test set for a very rare category...","[""When building a classifier for a very rare c...",1,0,0
4,198779,koala-13b,gpt-3.5-turbo-0314,"[""What is the best way to travel from Tel-Aviv...","[""The best way to travel from Tel Aviv to Jeru...","[""The best way to travel from Tel-Aviv to Jeru...",0,1,0
...,...,...,...,...,...,...,...,...,...
57472,4294656694,gpt-4-0613,claude-1,"[""A simple mnemonic for \u03c0:\n\""How I wish ...","[""Sure, let's break it down:\n\n1. \""How\"" has...","[""Here is how that mnemonic represents the dig...",1,0,0
57473,4294692063,claude-2.0,llama-2-13b-chat,"[""In python, implement a naive Bayes with gaus...","[""Here is an implementation of a naive Bayes c...","[""Sure! Here's an implementation of a naive Ba...",1,0,0
57474,4294710549,claude-1,alpaca-13b,"[""is it unethical to work on building weapons?...","[""Working on weapons technology raises some et...","[""It depends on the context. Weapons can be us...",1,0,0
57475,4294899228,palm-2,tulu-2-dpo-70b,"[""If a bait contains 0,0025% bromadiolon then ...","[""Bromadiolone is a rodenticide which is most ...","[""As an AI language model, I do not promote or...",0,1,0


In [33]:
from tqdm import tqdm
import json
import random

In [45]:
counter = {1: 0, 2: 0, 3: 0}
one_turn = []
for i in tqdm(range(len(df))):
    x = df.iloc[i]
    question = json.loads(x['prompt'])
    answer_a = json.loads(x['response_a'])
    answer_b = json.loads(x['response_b'])
    assert len(question) == len(answer_a)
    assert len(answer_a) == len(answer_b)
    n = len(question)
    if n == 1 or n == 2:
        counter[n] += 1
    else:
        counter[3] += 1
    if n == 1:
        one_turn.append({
            'id': int(x['id']),
            'model_a': x['model_a'],
            'model_b': x['model_b'],
            'winner_model_a': int(x['winner_model_a']),
            'winner_model_b': int(x['winner_model_b']),
            'winner_tie': int(x['winner_tie']),
            'question': question[0],
            'answer_a': answer_a[0],
            'answer_b': answer_b[0],
        })

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57477/57477 [00:03<00:00, 14724.55it/s]


In [46]:
with open("lmsys-chatbot-arena/train.single-turn.json", "w") as f:
    json.dump(one_turn, f, indent=4)

In [47]:
len(one_turn)

49938

In [24]:
counter

{1: 49938, 2: 4673, 3: 2866}

In [16]:
question

['Is it morally right to try to have a certain percentage of females on managerial positions?',
 'OK, does pineapple belong on a pizza? Relax and give me fun answer.']

In [18]:
len(answer_a)

2

In [19]:
len(answer_b)

2

In [28]:
x['winner_model_a']

1

# Sainity Check

In [48]:
with open("lmsys-chatbot-arena/train.single-turn.json") as f:
    data = json.load(f)

In [50]:
winners = {'A': 0, 'B': 0, 'tie': 0}

In [51]:
for i in tqdm(range(len(data))):
    x = data[i]
    assert x['winner_model_a'] + x['winner_model_b'] + x['winner_tie'] == 1
    if x['winner_model_a'] == 1:
        winner_gt = "A"
    elif x['winner_model_b'] == 1:
        winner_gt = "B"
    elif x['winner_tie'] == 1:
        winner_gt = "tie"
    winners[winner_gt] += 1

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49938/49938 [00:00<00:00, 1505452.80it/s]


In [53]:
17312 / 49938 * 100

34.66698706395931

In [54]:
16985 / 49938 * 100

34.012175097120426

In [55]:
15641 / 49938 * 100

31.320837838920262